In [10]:
import xarray as xr
import netCDF4
import numpy as np
import pandas as pd
import aasi 
import pyeto as pt

In [13]:
#User-supplied

mapfile = '/glade/p/work/manab/ff/forcinggeneration/mappingislandpark.nc'
forcfile = '/glade/p/work/manab/ff/forcinggeneration/islandparkforc/ens_forc.PNW.0625.1980-1989.051.nc'


def calcWeightedAvg(ncdat, varname):
    '''
    Calculates the weighted average 
    of any <varname> in the mapping file
    '''
    wAvg = sum(ncdat[varname]*ncdat['weight'])
    return(wAvg)

def calcPET(lat, time, tmin, tmax, tmean):
    '''
    Calculates Potential Evapotranspiration using 
    Hargreaves equation (Hargreaves and Samani, 1985) 
    '''
    
    latrad = pt.deg2rad(lat)                             #Latitude to radians
    dayofyear = pd.Series(time).dt.day.values
    etrad = []
    pet = []
    
    #Calculate ET radiation
    for x in np.nditer(dayofyear):
        soldec = pt.sol_dec(x)                           #Solar declination
        sha = pt.sunset_hour_angle(latrad, soldec)       #Sunset hour aingle
        ird = pt.inv_rel_dist_earth_sun(x)               #Inverse relative distance Earth-Sun
        etrad.append(pt.et_rad(latrad, soldec, sha, ird))#Extraterrestrial radiation
    
    #Calculate PET using hargreaves
    for x in range(0, len(etrad)):
        pet.append(pt.hargreaves(tmin[x], tmax[x], tmean[x], etrad[x]))
    
    pet = np.array(pet)
    return(pet)                

if __name__ == '__main__':
    
    mapdat = xr.open_dataset(mapfile)
    forcdat = xr.open_dataset(forcfile)
    
    lat = calcWeightedAvg(mapdat, 'latitude')
    lon = calcWeightedAvg(mapdat, 'longitude')
    
    #Extracts the variables values from forcing file
    prcp = forcdat['prcp'].values
    tmax = forcdat['tmax'].values
    tmin = forcdat['tmin'].values
    time = forcdat['time'].values
    tmean = (tmax+tmin)/2
    
    pet = calcPET(lat, time, tmin, tmax, tmean)

    prcp = np.reshape(prcp,(prcp.shape[0],1,1))
    pet = np.reshape(pet,(pet.shape[0],1,1))
    tmean = np.reshape(tmax,(tmean.shape[0],1,1))
    
    #Create new netCDF file
    f = xr.Dataset({'pr': (['time','latitude', 'longitude'],  prcp),
                    'pet': (['time','latitude', 'longitude'],  pet),
                    'temp': (['time','latitude', 'longitude'],  tmean)
                   },
                   coords={'time': time,
                           'latitude':[lat],
                           'longitude':[lon]}
                  )      
    

In [14]:
f

<xarray.Dataset>
Dimensions:    (latitude: 1, longitude: 1, time: 3653)
Coordinates:
  * time       (time) datetime64[ns] 1980-01-01 1980-01-02 1980-01-03 ...
  * latitude   (latitude) float64 44.51
  * longitude  (longitude) float64 -111.4
Data variables:
    pr         (time, latitude, longitude) float32 0.974603 1.94124 2.28753 ...
    pet        (time, latitude, longitude) float32 0.430324 0.365902 ...
    temp       (time, latitude, longitude) float32 0.602832 -1.47469 ...